# Memory

Describing memory and getting it synthesized correcly is cumbersome.
Therefore, this will only demonstrate a simple approach to generate memory using the intermediate representation language dialect (IRL).
This modification enables shared variable output for VHDL to support true dual port behaviour.
The general strategy with the MyIRL synthesis is to use a blackbox element (rely on the library)

In [1]:
from myhdl import intbv
from myirl import *
import random

import sys
sys.path.append('../')


<frozen importlib._bootstrap>:219: RuntimeWarning: myirl.kernel.convertible.BinOp size changed, may indicate binary incompatibility. Expected 16 from C header, got 32 from PyObject
<frozen importlib._bootstrap>:219: RuntimeWarning: myirl.kernel.convertible.BoolOp size changed, may indicate binary incompatibility. Expected 16 from C header, got 32 from PyObject
<frozen importlib._bootstrap>:219: RuntimeWarning: myirl.kernel.convertible.Eq size changed, may indicate binary incompatibility. Expected 16 from C header, got 32 from PyObject
<frozen importlib._bootstrap>:219: RuntimeWarning: myirl.kernel.convertible.And size changed, may indicate binary incompatibility. Expected 16 from C header, got 48 from PyObject
<frozen importlib._bootstrap>:219: RuntimeWarning: myirl.kernel.convertible.Or size changed, may indicate binary incompatibility. Expected 16 from C header, got 48 from PyObject
<frozen importlib._bootstrap>:219: RuntimeWarning: myirl.kernel.convertible.Not size changed, may indi

## Dual port memories

We can not use a `SigArray` for the RAM, because a true dual port RAM would provoke unresolved multiple drivers.
Therefore we hack the `myirl.lists.SigArray` class by deriving and choosing a different element in return of the `__getitem__` member which uses a shared variable.

**Note** Not working in VHDL-2008

In [2]:
from myirl import lists
from myirl.kernel import extensions, utils

from myirl import targets

class CellAssign:
    def __init__(self, parent, portion, val, width):
        self._vref = parent
        self.portion = portion
        self.value = val
        self.width = width
        
    def emit(self, context):
        tgt = context.target
        if tgt.lang != 'VHDL':
            raise TypeError("Unsupported target %s" % type(tgt))
        n, v = self._vref.identifier, self.value
        p = self.portion
        sz = self.width
        context.output("%s(to_integer(%s)) := %s;\n" % (n, p, base.convert(v, tgt, sz)))        

class ArrayElem(lists.SigArrayElem):
    decl_type_vhdl = "shared variable"
    
    def set(self, val):
        w = self.default().size()
        return CellAssign(self.seq, self.index, val, w)

class RamBuffer(lists.SigArray):    
    def __getitem__(self, item):
        if isinstance(item, (Sig, int)):
            # We can not just return self.val[item], as the iterator
            # has not initialized yet.
            return ArrayElem(self, item)
        else:
            raise TypeError("Multi item slicing of iterator not supported")

In [3]:
class RamPort:
    __slots__ = ('clk', 'we', 'ra', 'wa', 'rd', 'wd')
    def __init__(self, AWIDTH, DWIDTH):
        self.clk = ClkSignal()
        self.we = Signal(bool())
        self.ra, self.wa = [ Signal(intbv()[AWIDTH:]) for i in range(2) ]
        self.rd, self.wd = [ Signal(intbv()[DWIDTH:]) for i in range(2) ] 
        
@block
def tdp_ram(pa, pb, INITDATA):
    inst = []
 
    def gen_logic(p, i):
        "Generate port mechanics inline"
        @process(p.clk, EDGE=p.clk.POS)
        def proc(_logic):
            _logic += [
                _logic.If(p.we == True).Then(
                    buf[p.wa].set(p.wd)
                ),
                p.rd.set(buf[p.ra])
            ]
        proc.rename("proc%d" % i)
        return proc

    buf = RamBuffer(INITDATA)
    
    for i, p in enumerate([pa, pb]):
        inst.append(gen_logic(p, i))
    
    return inst

In [4]:
from myirl import targets
from myirl.test.common_test import run_ghdl

random.seed(0)

def test():
    RAM_CONTENT = [ intbv(random.randint(0, 2 ** 9))[8:] for i in range(2 ** 9) ]
    pa, pb = [ RamPort(AWIDTH=9, DWIDTH=8) for i in range(2) ]
    inst = tdp_ram(pa, pb, RAM_CONTENT)
    
    f = inst.elab(targets.VHDL)
    run_ghdl(f, inst, std = "93", debug = True) # Note we run with std '93'

In [5]:
test()

Creating process 'gen_logic/proc' with sensitivity (<pa_clk>,)
Creating process 'gen_logic/proc' with sensitivity (<pb_clk>,)
 Insert unit tdp_ram/__RamPort____RamPort____list__ 
 DEBUG: Writing 'tdp_ram' to file /tmp/tdp_ram.vhdl 
Finished _elab in 0.0027 secs
==== COSIM stdout ====

==== COSIM stderr ====

==== COSIM stdout ====
analyze /home/testing/.local/lib/python3.8/site-packages/myirl-0.0.0-py3.8-linux-x86_64.egg/myirl/targets/../test/vhdl/txt_util.vhdl
analyze /home/testing/.local/lib/python3.8/site-packages/myirl-0.0.0-py3.8-linux-x86_64.egg/myirl/targets/libmyirl.vhdl
analyze /tmp/tdp_ram.vhdl
elaborate tdp_ram

==== COSIM stderr ====

==== COSIM stdout ====

==== COSIM stderr ====



In [6]:
! grep -A 20 MyIRL /tmp/tdp_ram.vhdl 

architecture MyIRL of tdp_ram is
    -- Local type declarations
    -- Signal declarations
    type a_s_1187 is array (0 to 511) of unsigned(7 downto 0);
    shared variable s_1187 : a_s_1187 := (
        x"8a",x"ae",x"29",x"09",x"f1",x"9e",x"36",x"e8",x"6e",x"df",
        x"8e",x"20",x"8f",x"61",x"00",x"96",x"3d",x"65",x"4b",x"52",
        x"e3",x"67",x"6a",x"bc",x"43",x"d1",x"e8",x"c5",x"0a",x"3f",
        x"0e",x"5f",x"98",x"01",x"f9",x"55",x"f9",x"4d",x"40",x"c3",
        x"e3",x"f4",x"91",x"ca",x"5d",x"52",x"47",x"f5",x"6f",x"34",
        x"2a",x"7f",x"54",x"d0",x"26",x"c7",x"5d",x"8a",x"44",x"f7",
        x"29",x"bc",x"c1",x"bf",x"21",x"0a",x"e7",x"46",x"5b",x"85",
        x"99",x"27",x"52",x"90",x"1a",x"f1",x"dc",x"ad",x"19",x"cd",
        x"f8",x"6d",x"54",x"4c",x"76",x"f2",x"57",x"c2",x"f8",x"10",
        x"15",x"77",x"e1",x"7c",x"ae",x"54",x"b4",x"3f",x"67",x"95",
        x"e0",x"2e",x"4b",x"1b",x"7f",x"c1",x"7a",x"90",x"5d",x"7b",
        x"76",x"25",x"16",x"c7",x"bd",x"7e",